In [1]:
#export
import k1lib, json, base64, threading; import k1lib.cli as cli
from collections import deque

Kinda like _basics, but here, it's assumed that the environment is fully loaded, so I can use any functionality, not just pure Python alone

In [2]:
#export
__all__ = ["log", "aes_encrypt", "aes_decrypt"]

In [3]:
#export
_logObj = {"loaded": False, "logMsgs": deque(), "path": None}
def _thTarget():
    import asyncio, base64, json; from k1lib import kws
    async def main():
        async with kws.WsClient("wss://ws.logs.mlexps.com/_k1_ingest") as ws:
            while True:
                if len(_logObj["logMsgs"]) == 0: await asyncio.sleep(0.01)
                else: await ws.send(_logObj["logMsgs"].popleft())
    asyncio.new_event_loop().run_until_complete(main())
def log(path:str, obj:"any"):
    """Logs random debug statements to logs.mlexps.com server.
Example::

    k1.log("ggdrive/topic1", "some message")
    k1.log("ggdrive/topic1/sub2", {"some": "json", "object": 2})
    
    # I typically do it like this, so that I can filter down only the messages that I want based on severity
    k1.log("ggdrive/info", {"some": "json", "object": 2})
    k1.log("ggdrive/error", {"some": "json", "object": 2})

Visit the website https://logs.mlexps.com/watch/ggdrive, or
/watch/ggdrive/topic1, or /watch/ggdrive/topic1/sub2 to view all logs
coming in."""
    if not _logObj["loaded"]: _logObj["loaded"] = True; threading.Thread(target=_thTarget).start()
    if not isinstance(obj, (str, float, int)):
        obj = base64.b64encode(json.dumps(obj).encode()).decode()
    _logObj["logMsgs"].append(f"{path}/{obj}")

In [4]:
#export
if k1lib.settings.startup.import_optionals:
    try:
        from scipy import stats
        __all__.append("pValue")
        def pValue(zScore):
            """2-sided p value of a particular z score. Requires :mod:`scipy`."""
            return stats.norm.sf(abs(zScore))*2
    except: pass

In [53]:
#export
try:
    Crypto = k1lib.dep("Crypto", "pycryptodome", url="https://pycryptodome.readthedocs.io/en/latest/")
    def aes_encrypt(plaintext:bytes) -> str: Crypto.Cipher
    def aes_decrypt(plaintext:bytes) -> str: Crypto.Cipher
    from Crypto.Cipher import AES
    from Crypto.Random import get_random_bytes
    from Crypto.Util.Padding import pad, unpad
    def aes_encrypt(plaintext:bytes) -> str:
        """Encrypts a message using AES.
Example::

    res = k1lib.aes_encrypt(b"some message") # can return '3HV7PKKQL2DLWQWBBTETQTXNMC4Q6DJ2FSS73A7NCRAX6K4ZZKXQ===='
    k1lib.aes_descrypt(res) # returns b"some message"

After encrypting, this is encoded using base32, ready to be used in urls. This function
is a convenience function meant for small messages here and there, and is not intended
for heavy duty encryption.

The key is automatically generated, and is configurable via `settings.cred.aes.key`
"""
        if not isinstance(plaintext, bytes): plaintext = f"{plaintext}".encode()
        cipher = AES.new(k1lib.settings.cred.aes.key, AES.MODE_CBC); ciphertext = cipher.encrypt(pad(plaintext, AES.block_size))
        return base64.b32encode(cipher.iv + ciphertext).decode()
    def aes_decrypt(ciphertext:str) -> bytes:
        """Decrypts a message using AES.
See :meth:`aes_encrypt` for more information."""
        ciphertext = base64.b32decode(ciphertext.encode()); iv = ciphertext[:AES.block_size]; cipher = AES.new(k1lib.settings.cred.aes.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(ciphertext[AES.block_size:]), AES.block_size)
    k1lib.settings.cred.add("aes", k1lib.Settings().add("key", get_random_bytes(16), "16-byte aes key, used in aes_encrypt() and aes_decrypt()", sensitive=True), "anything related to AES block cipher")
except: pass

In [49]:
assert aes_decrypt(aes_encrypt(b"some message")) == b"some message"

In [55]:
!../export.py _advanced --upload=True

2024-04-04 16:36:29,326	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-04-04 16:36:29,350	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15817   0   61%   
10195   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [50]:
!../export.py _advanced

2024-04-04 16:32:21,081	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.17:6379...
2024-04-04 16:32:21,110	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
15815   0   61%   
10191   1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.7
Uninstalling k1lib-1.7:
  Successfully uninstalled k1lib-1.7
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg

In [1]:
!../export.py _advanced --bootstrap=True

Traceback (most recent call last):
  File "/home/kelvin/repos/labs/k1lib/k1lib/../export.py", line 10, in <module>
    try: from k1lib.imports import *; hasK1 = True
  File "/home/kelvin/repos/labs/k1lib/k1lib/__init__.py", line 5, in <module>
    from ._basics import *
  File "/home/kelvin/repos/labs/k1lib/k1lib/_basics.py", line 260, in <module>
    if k1lib.settings.startup.import_optionals:                                      # scaleSvg
AttributeError: partially initialized module 'k1lib' has no attribute 'settings' (most likely due to a circular import)
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py 